In [31]:
from xml.etree.ElementTree import parse
import re
# 不处理test，拆分train
path='test'
data1 = parse(path+'.xml')
root=data1.getroot()
sentiment2id = {'negative':'1','neutral':'2','positive':'3'}

def get_text(root):
    with open(path+'.tsv','w',encoding='gbk') as f:
        f.write('labels\tkey_words\treview\n')
        for Review in root.iter('Reviews'):
            review = Review.findall('Review')
            for each in review:
                for each_sentence in each.iter('sentence'):

                    text = each_sentence.find('text').text
                    text = re.sub('"',"",text)
                    Opinions = each_sentence.find('Opinions')
                    for each_opinion in Opinions.findall('Opinion'):
                        target =  each_opinion.get('target')
                        category = each_opinion.get('category')
                        polarity = each_opinion.get('polarity')
                        f.write(sentiment2id[polarity]+category+'\t'+target+'\t'+'"'+text+'"'+'\n')

get_text(root)

In [33]:
import pandas as pd
import numpy as np
ALL_cats=[]
path1='test'
path2='train' if path1=='test' else 'test'
data1=pd.read_csv(path1+'.tsv',delimiter='\t',encoding='gbk')
data1.columns=['1','2','3']
data1['senti']=data1['1'].apply(lambda x:x[0])
data1['CAT']=data1['1'].apply(lambda x:x[1:])
data1['CAT1']=data1['CAT'].apply(lambda x:x.split('#')[0])
data1['CAT2']=data1['CAT'].apply(lambda x:x.split('#')[1])
data1['KEYWORDS']=data1['2']
data1['review']=data1['3']
data1=data1[['senti','CAT1','CAT2','KEYWORDS','review']]
data1_=pd.DataFrame(columns=['CAT','KEYWORDS','review'])
for i in range(len(data1)):
    review=data1['review'][i]
    cat=[data1['CAT1'][i],data1['CAT2'][i]]
    ALL_cats.extend(cat)
    senti=data1['senti'][i]
    keywords=data1['KEYWORDS'][i] if type(data1['KEYWORDS'][i])==str else 'NEG'
    for j in cat:
        data1_=data1_.append(
            pd.DataFrame(
                [[j,str(senti)+' '+keywords,review]],
                columns=['CAT','KEYWORDS','review']
            )
        )

data2=pd.read_csv(path2+'.tsv',delimiter='\t',encoding='gbk')
data2.columns=['1','2','3']
data2['senti']=data2['1'].apply(lambda x:x[0])
data2['CAT']=data2['1'].apply(lambda x:x[1:])
data2['CAT1']=data2['CAT'].apply(lambda x:x.split('#')[0])
data2['CAT2']=data2['CAT'].apply(lambda x:x.split('#')[1])
data2['KEYWORDS']=data2['2']
data2['review']=data2['3']
data2=data2[['senti','CAT1','CAT2','KEYWORDS','review']]
data2_=pd.DataFrame(columns=['CAT','KEYWORDS','review'])
for i in range(len(data2)):
    review=data2['review'][i]
    cat=[data2['CAT1'][i],data2['CAT2'][i]]
    ALL_cats.extend(cat)
    senti=data2['senti'][i]
    keywords=data2['KEYWORDS'][i] if type(data2['KEYWORDS'][i])==str else 'NEG'
    for j in cat:
        data2_=data2_.append(
            pd.DataFrame(
                [[j,str(senti)+' '+keywords,review]],
                columns=['CAT','KEYWORDS','review']
            )
        )

classes=list(set(ALL_cats))
# 排序
for each in classes:
    try:
        print(each)
    except:
        pass

with open('label.txt','w') as f:
    for each in classes:
        f.write(each+'\n')
data1_.to_csv('关键词抽取结果_'+path1+'.csv',index=False,header=['labels','key_words','review'])
data2_.to_csv('关键词抽取结果_'+path2+'.csv',index=False,header=['labels','key_words','review'],encoding='utf-8')

C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1_=data1_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:23: FutureWarning: The frame.append method is 

STYLE_OPTIONS
DRINKS
MISCELLANEOUS
PRICES
GENERAL
QUALITY
AMBIENCE
SERVICE
RESTAURANT
FOOD
LOCATION


C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data2_=data2_.append(
C:\Users\93461\AppData\Local\Temp\ipykernel_56932\66465054.py:47: FutureWarning: The frame.append method is 

In [10]:
# 数据增强
import pandas as pd
data = pd.read_csv('关键词抽取结果_train.csv')
data['index2']=data['key_words'].apply(lambda x:x[0])
data1  = data[data['index2']=='2']

data=data.append([data1]*5,ignore_index=True)
data.drop('index2',axis=1,inplace=True)

data.to_csv('关键词抽取结果_train_aug.tsv',index=False,header=None)

print(data['key_words'].apply(lambda x:x[0]).value_counts())

3    2228
1    1032
2     936
Name: key_words, dtype: int64


C:\Users\93461\AppData\Local\Temp\ipykernel_62908\447091782.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append([data1]*5,ignore_index=True)
